In [ ]:
using Pkg
Pkg.add("DelimitedFiles")

In [ ]:
using TSSOS
using DynamicPolynomials
using Random
using Statistics
using DataFrames
using DelimitedFiles

In [16]:
function observation(T,ts,start)
    # randomly select a trajectory of time series
    return copy(ts[start:start+T-1])
end

observation (generic function with 1 method)

In [ ]:
function parameter_estimation2(Y)
    # training process
    # use Y to learn the system parameters in f[t+1] = Fdash*f[t] + p[t]
    # output Fdash and p[1:T]
    
    T=length(Y)
    
    @polyvar G p[1:T] q[1:T-1] f[1:T+1] m[1:T+1]; # all variables are assumed to be nc
    var=vcat(m,f,p,q,G);

    # constraints
    ine1 = [f[i] - m[i] - p[i] for i in 1:T];
    ine2 = [- f[i] + m[i] + p[i] for i in 1:T];
    ine3 = [m[i+1] - G*m[i] - q[i] for i in 1:T-1];
    ine4 = [-m[i+1] + G*m[i] + q[i] for i in 1:T-1];
    ine5 = [m[T+1] - G*m[T],-m[T+1] + G*m[T]]
    #ine6 = [f[T+1] - m[T+1],-f[T+1] + F*m[T+1]]

    #objective
    obj=sum((Y[i]-f[i])^2 for i in 1:T)+ 0.01*sum(p[i]^2 for i in 1:T) +0.01*sum(q[i]^2 for i in 1:T-1) #+ p[T+1]^2 + q[T]^2 #+ 0.0001*sum(p[i]^2 for i in 1:T); #+0.1*q[i]^2

    # pop
    pop=vcat(obj,ine1,ine2,ine3,ine4,ine5); #,ine5,ine6

    # solve model 
    opt,sol,data=tssos_first(pop,var,1,TS="MD",solution=true);
    print(sol)
    return sol[T+1] 
end

In [ ]:
function rmse(Y_predict, Y_true)
    # compare predicted Y_[T+1：T+pred] with actual value of predicted Y_[T+1：T+pred]
    # calcluate rmse: sqrt( sum((Y-f)^2)/N) 
    Y=copy(Y_true[1:length(Y_predict)])
    return  abs(Y[1]-Y_predict[1])
    #1-sum( abs(Y[i]-Y_predict[i]) for i in 1:length(Y_predict) )/ sum(abs(Y[i]-mean(Y)) for i in 1:length(Y_predict) )
end

In [15]:
# read observations
# the stock-market.txt is generated by
"""
# Load stock-market data
load_path = 'setting6.mat'
load_data = sio.loadmat(load_path)
seq=flatten(load_data['seq_d0'].tolist())
"""
ts = readdlm( "stock-market.txt" );

1

In [19]:
## a single run (because the first time using TSSOS is very slow)
T=20 # the length of time window is 20
pred=1 # we are making one-step ahead prediction
Y=observation(T+pred,ts,1) # select a 21-period time series from stock-market data. 
parameter_estimation2(Y[1:T]) # system identification using the first 20-period

*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2022
TSSOS is launching...


Starting to compute the block structure...


-----------------------------------------------------------------------------
The sizes of PSD blocks:
[3, 2, 1]
[20, 60, 1]
-----------------------------------------------------------------------------


Obtained the block structure in 2.0909376 seconds. The maximal size of blocks is 3.


Assembling the SDP...
There are 3486 affine constraints.


...

18  3.6e-08  2.3e-07  3.5e-19  1.00e+00   3.880635900e-05   3.880354181e-05   1.0e-14  50.44 
Optimizer terminated. Time: 50.47   

SDP solving time: 56.1599317 seconds.


optimum = 3.880635900302998e-5



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Found a locally optimal solution by Ipopt, giving an upper bound: 0.005739290565543342 and a relative optimality gap: 0.5700484206540313%.


[31.128223451830724, 30.82781419964467, 30.692154371569156, 30.738113646335112, 30.699046290024697, 30.853271957586863, 30.93886360636541, 30.821074782018382, 30.729595179679915, 30.710479725934675, 30.7448042140443, 30.86654968029453, 30.981612708106596, 31.0477829345383, 31.067208878311316, 31.054172145944495, 31.15188852149376, 31.276620711460982, 31.31771925086907, 31.193904190639717, 31.197208875453484, 31.431962608433974, 30.663047665343015, 30.51051637991652, 30.823149640062738, 30.505733131584396, 30.921913583738483, 31.14226597630065, 30.79476311665365, 30.657223714650296, 30.657034452732027, 30.657374299148955, 30.873233165151436, 31.030511016911934, 31.094532504302364, 31.09967533542882, 30.943407645009348, 31.12487018338113, 31.360263571400612, 31.48265068565217, 31.066771328620202, 2.1860799002178702e-15, 0.30373915660327555, -0.16476653430162877, -0.18163799165260777, 0.085035993727653, -0.19331315844027738, 0.06864162615164174, 0.20340236993526625, -0.026311665364704123,

31.197208875453484

In [ ]:
T=20
pred=1

opt=DataFrame(time=[],pred=[],rmse=[])
for s in 1:100
    Y=observation(T+pred,ts,s);
    Y_predict=parameter_estimation2(Y[1:T]);
    opt_tem=rmse(Y_predict,Y[T+1:T+pred]); # assess the prediction 
    push!(opt,(s,Y_predict,opt_tem))
end

In [ ]:
CSV.write("stock_TSSOS_d1.csv", opt)

In [ ]:
opt